In [1]:
import torch

In [4]:
# model = torch.hub.load("harvard-visionlab/modelhub", "alexnet_pytorch")
# model

In [8]:
import os
import torch

model_dir = os.path.join(torch.hub.get_dir(), "checkpoints")
url = 'https://download.pytorch.org/models/alexnet-owt-7be5be79.pth'
cache_filename = 'alexnet-owt-7be5be79.pth'

checkpoint = torch.hub.load_state_dict_from_url(
    url = url,
    model_dir = model_dir,
    map_location = 'cpu',
    progress = True,
    check_hash = True,
    file_name = cache_filename
)

r = torch.hub.HASH_REGEX.search(cache_filename)  # r is Optional[Match[str]]
hashid = r.group(1) if r else None
weights_file = os.path.join(model_dir, cache_filename)
hashid, weights_file

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/jovyan/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

('7be5be79',
 '/home/jovyan/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth')

In [9]:
ls {model_dir}

alexnet-owt-7be5be79.pth


In [ ]:
# pip install git+ssh://git@github.com/harvard-visionlab/viionlab_models.git

In [7]:
import os
default_model_dir = os.path.join(torch.hub.get_dir(), "checkpoints")
default_model_dir

'/home/jovyan/.cache/torch/hub/checkpoints'

In [31]:
import boto3
from botocore.exceptions import ClientError

def download_s3_url_to_file(url, dst, hash_prefix=None, progress=True, profile=None, endpoint_url=None):
    r"""Download object at the given S3 URL to a local path.
    Args:
        url (str): S3 URL of the object to download, e.g. s3://my-bucket/my-file.pth
        dst (str): Full path where object will be saved, e.g. ``/tmp/temporary_file``
        hash_prefix (str, optional): If not None, the SHA256 downloaded file should start with ``hash_prefix``.
            Default: None
        progress (bool, optional): whether or not to display a progress bar to stderr
            Default: True
    Example:
        >>> torch.hub.download_s3_url_to_file('s3://my-bucket/my-file.pth', '/tmp/temporary_file')
    """
    bucket, key = url.replace("s3://", "").split("/", 1)
    # print(bucket, key)
    file_size = None
    # s3 = boto3.client('s3')
    session = boto3.Session(profile_name=profile)
    s3 = session.client('s3', endpoint_url=endpoint_url)

    try:
        obj = s3.head_object(Bucket=bucket, Key=key)
        file_size = obj['ContentLength']
    except ClientError as e:
        print(e)
        return

    # We deliberately save it in a temp file and move it after
    # download is complete. This prevents a local working checkpoint
    # being overridden by a broken download.
    dst = os.path.expanduser(dst)
    dst_dir = os.path.dirname(dst)
    f = tempfile.NamedTemporaryFile(delete=False, dir=dst_dir)

    try:
        if hash_prefix is not None:
            sha256 = hashlib.sha256()
        with tqdm(total=file_size, disable=not progress,
                  unit='B', unit_scale=True, unit_divisor=1024) as pbar:
            response = s3.get_object(Bucket=bucket, Key=key, RequestPayer='requester')
            while True:
                buffer = response['Body'].read(8192)
                if len(buffer) == 0:
                    break
                f.write(buffer)
                if hash_prefix is not None:
                    sha256.update(buffer)
                pbar.update(len(buffer))

        f.close()
        if hash_prefix is not None:
            digest = sha256.hexdigest()
            if digest[:len(hash_prefix)] != hash_prefix:
                raise RuntimeError('invalid hash value (expected "{}", got "{}")'
                                   .format(hash_prefix, digest))
        shutil.move(f.name, dst)
    finally:
        f.close()
        if os.path.exists(f.name):
            os.remove(f.name)


In [35]:
url = "s3://visionlab-models/alexnet_pytorch-category_supervised-imagenet1k/alexnet-owt-7be5be79.pth"
model_dir = '../weights'
filename = 'alexnet-owt-7be5be79.pth'
cached_file = os.path.join(model_dir, filename)
hash_prefix = '7be5be79'
progress = True
profile = 'wasabi'
endpoint_url = 'https://s3.wasabisys.com'
download_s3_url_to_file(url, cached_file, hash_prefix, progress=progress, 
                        profile=profile, endpoint_url=endpoint_url)

100%|██████████| 233M/233M [00:10<00:00, 22.8MB/s] 


In [24]:
# download_url_to_file(url, dst, hash_prefix=None, progress=True, profile=None, endpoint_url=None)

In [34]:
from botocore import UNSIGNED

session = boto3.Session(profile_name='wasabi')
s3_client = session.client('s3', endpoint_url='https://s3.wasabisys.com')

bucket_name = 'visionlab-models'

# List the contents of the bucket
response = s3_client.list_objects_v2(Bucket=bucket_name)

# Print the names of all the objects in the bucket
for obj in response['Contents']:
    print(obj['Key'])    

alexnet_pytorch-category_supervised-imagenet1k/
alexnet_pytorch-category_supervised-imagenet1k/alexnet-owt-7be5be79.pth
